In [1]:
from pathlib import Path
import tensorflow as tf, pickle
from datetime import datetime
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt    

2025-06-17 16:22:56.710390: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750170176.756489   56864 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750170176.771933   56864 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750170176.798734   56864 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750170176.798786   56864 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750170176.798789   56864 computation_placer.cc:177] computation placer alr

In [2]:

# ── 1 · load model (architecture + weights already inside) ────────────
save_dir   = Path("dfs training")
ticker     = "AAPL"
model_path = save_dir / f"model_{ticker}_2025-06-17.keras"   # adjust date
model      = tf.keras.models.load_model(model_path)

# (optional) re-load weights-only checkpoint in case you updated it
ckpt_path  = save_dir / f"best_{ticker}.weights.h5"
model.load_weights(ckpt_path)      # harmless if identical

# ── 2 · load tf.data test set exactly as it was saved ─────────────────
spec      = pickle.loads((save_dir / f"ds_test_{ticker}.pkl").read_bytes())
ds_test   = tf.data.Dataset.load(str(save_dir / "ds_test_data"),
                                 element_spec=spec,
                                 compression="GZIP")



ValueError: File not found: filepath=dfs training/model_AAPL_2025-06-17.keras. Please ensure the file is an accessible `.keras` zip file.

In [ ]:
# ── 3 · evaluate RMSE day-by-day (stateful) ───────────────────────────
lstm_layers = [l for l in model.layers if hasattr(l, "reset_states")]
test_metric = tf.keras.metrics.RootMeanSquaredError()

N_test = sum(1 for _ in ds_test)     # concrete length for tqdm
ds_test = ds_test.cache()

for X_day, y_day, _ in tqdm(ds_test, total=N_test, unit="day", desc="Testing"):
    for l in lstm_layers: l.reset_states()          # flush hidden state
    y_pred = model(X_day, training=False)
    test_metric.update_state(y_day, y_pred)

print(f"\nFINAL OOS TEST RMSE = {test_metric.result().numpy():.5f}")



In [ ]:
# ── 4 · quick visual sanity check on first N_PLOTS sessions ───────────
N_PLOTS, plotted = 10, 0
for X_day, y_day, _ in ds_test:
    for l in lstm_layers: l.reset_states()
    y_hat = model(X_day, training=False).numpy().squeeze()
    y_true = y_day.numpy().squeeze()

    plt.figure(figsize=(10, 3))
    plt.plot(y_true, label="y",   color="black", linewidth=1)
    plt.plot(y_hat,  label="pred",color="firebrick", alpha=0.8)
    plt.title(f"Test session #{plotted+1} • length = {len(y_true)} bars")
    plt.legend(); plt.grid(True); plt.show()

    plotted += 1
    if plotted >= N_PLOTS: break

In [ ]:
# ###############################################################################
# # 6. FINAL  EVALUATION  ON  THE  UNTOUCHED  TEST  SPLIT                       
# ###############################################################################

# # 1) Reload champion weights
# model.load_weights(ckpt_path)

# # 2) Grab all stateful layers once
# lstm_layers = [l for l in model.layers if hasattr(l, "reset_states")]

# # 3) Metric accumulator
# test_metric = tf.keras.metrics.RootMeanSquaredError()

# # 4) Concrete length → makes tqdm show “n / total” instead of “?/”
# N_test = sum(1 for _ in ds_test)      # cheap count
# ds_test = ds_test.cache()             # allows second pass if needed

# # 5) Iterate with progress bar
# for X_day, y_day, new_day in tqdm(ds_test,
#                                   total=N_test,
#                                   unit="day",
#                                   desc="Testing"):
#     # flush hidden state at calendar boundary
#     for l in lstm_layers: l.reset_states()

#     # forward pass (no gradients) + metric update
#     y_pred = model(X_day, training=False)
#     test_metric.update_state(y_day, y_pred)

# # 6) Final report
# print(f"\nFINAL OUT-OF-SAMPLE TEST RMSE = {test_metric.result().numpy():.5f}")


In [ ]:
# ###############################################################################
# # Visual sanity-check:  y_true  vs.  y_pred  for several test days
# ###############################################################################

# # 1)  ── reload champion weights ──────────────────────────────────────
# model.load_weights(ckpt_path)

# # 2)  ── pick how many sessions you want to plot ──────────────────────
# N_PLOTS = 10                              
# plot_count = 0

# # 3)  ── loop over test dataset (one element = one day) ───────────────
# for X_day, y_day, new_day in ds_test:
#     # (a) reset state at calendar boundary (always True in this dataset)
#     for l in model.layers:
#         if hasattr(l, "reset_states"):
#             l.reset_states()

#     # (b) forward pass → prediction
#     y_pred = model(X_day, training=False)          # shape (1 , T , 1)
#     y_pred = y_pred.numpy().squeeze()              # (T,)
#     y_true = y_day.numpy().squeeze()               # (T,)

#     # (c) quick line plot
#     plt.figure(figsize=(10, 3))
#     plt.plot(y_true, label="label  (y)",  color="black",  linewidth=1.0)
#     plt.plot(y_pred, label="prediction",  color="firebrick", alpha=0.8)
#     plt.title(f"Test session #{plot_count+1}  •  length = {len(y_true)} bars")
#     plt.legend()
#     plt.grid(True)
#     plt.show()

#     plot_count += 1
#     if plot_count >= N_PLOTS:
#         break
